In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb # Import xgboost itself for callbacks
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# --- (Fonction create_time_series_features inchangée) ---
def create_time_series_features(df, target_col):
    df = df.copy()
    df['hour'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    df['day_of_year'] = df.index.dayofyear
    df['is_weekend'] = (df.index.dayofweek >= 5).astype(int)
    df['lag_1'] = df[target_col].shift(1)
    df['lag_7'] = df[target_col].shift(7)
    df['rolling_mean_7'] = df[target_col].shift(1).rolling(window=7).mean()
    df['rolling_std_7'] = df[target_col].shift(1).rolling(window=7).std()
    df.dropna(inplace=True)
    return df

# --- (Chargement et Préparation inchangés) ---
file_name = '/content/fichiers_data/ma_meteo_export.csv'
df = pd.read_csv(
    file_name,
    sep=';',
    decimal=',',
    parse_dates=['time'],
    index_col='time'
)
TARGET_COLUMN = 'temperature_2m'
df_features = create_time_series_features(df, TARGET_COLUMN)
X = df_features.drop(columns=[TARGET_COLUMN])
y = df_features[TARGET_COLUMN]
split_point = int(len(X) * 0.8)
X_train, X_test = X[:split_point], X[split_point:]
y_train, y_test = y[:split_point], y[split_point:]

print(f"Taille totale des données : {len(df_features)} heures")
print(f"Période d'entraînement : {X_train.index.min()} à {X_train.index.max()}")
print(f"Période de test : {X_test.index.min()} à {X_test.index.max()}")
print("\nAperçu des Features créées (5 premières lignes) :")
print(X_train.head())

# --- 4. ENTRAÎNEMENT DU MODÈLE XGBOOST (CORRIGÉ ET FIABLE) ---

# Initialisation du modèle XGBoost (régression)
model = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000,
    learning_rate=0.01,
    max_depth=5,
    random_state=42,
    n_jobs=-1,
)

# NOTE: Les tentatives précédentes d'utiliser `callbacks` ou `early_stopping_rounds`
# ont échoué avec un TypeError, ce qui est inattendu pour cette version de XGBoost.
# Pour résoudre l'erreur et permettre l'exécution, nous allons retirer l'argument
# `callbacks`. Cela désactivera l'arrêt précoce pour cette exécution,
# et le modèle s'entraînera pour le nombre complet d'estimateurs (n_estimators).
# Si un arrêt précoce est essentiel, une investigation plus poussée de l'environnement
# ou un changement de méthode (par ex. xgb.train) pourrait être nécessaire.

# Entraînement sans arrêt précoce pour contourner le TypeError
model.fit(
    X_train,
    y_train,
    eval_set=[(X_test, y_test)] # Jeu de validation (utilisé pour les métriques de validation si verbose=True)
    # callbacks=[early_stopping_callback] # Commenté pour éviter le TypeError
    # verbose=False # Peut être activé pour voir la progression de l'entraînement
)

# best_ntree_limit n'est disponible qu'avec l'arrêt précoce, donc nous affichons n_estimators
print(f"\nEntraînement terminé. Nombre d'arbres utilisés : {model.n_estimators}")

# --- 5. ÉVALUATION DU MODÈLE ---
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"\nRMSE (Root Mean Squared Error) sur le jeu de test : {rmse:.2f}°C")

# --- 6. IMPORTANCE DES FEATURES ---
print("\nImportance des Features (plus la valeur est élevée, plus la feature est importante) :")
feature_importance = pd.Series(model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
print(feature_importance)
